In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up image data generator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Load images of healthy and stressed plants
healthy_images = []
stressed_images = []
cnt, cnt2 = 0, 0
for file in os.listdir("C:/Users/styli/Downloads/RGB_Augmented/DRhealthy"):
    img = cv2.imread("C:/Users/styli/Downloads/RGB_Augmented/DRhealthy/" + file)
    healthy_images.append(img)
    if cnt == 150:
        break
    cnt += 1
for file in os.listdir("C:/Users/styli/Downloads/RGB_Augmented/DRstressed"):
    img = cv2.imread("C:/Users/styli/Downloads/RGB_Augmented/DRstressed/" + file)
    stressed_images.append(img)
    if cnt2 == 150:
        break
    cnt2 += 1
print(cnt, cnt2)

# Combine healthy and stressed plant images
images = healthy_images + stressed_images

# Create labels for the images
labels = []
for _ in range(len(healthy_images)):
    labels.append("healthy")
for _ in range(len(stressed_images)):
    labels.append("stressed")

# Encode labels as integers
le = LabelEncoder()
y = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=0.2)

# Generate batches of augmented training data
train_generator = data_generator.flow(X_train, y_train, batch_size=32)

# Train a random forest classifier
clf = RandomForestClassifier()
for epoch in range(10):
    print("Epoch: ", epoch+1)
    for i in range(10):
        X_batch, y_batch = next(train_generator)
        X_batch_reshaped = X_batch.reshape(X_batch.shape[0], -1)
        clf.partial_fit(X_batch_reshaped, y_batch, classes=np.unique(y))

# Make predictions on the test data
test_generator = data_generator.flow(X_test, y_test, batch_size=32)
X_test_batch, y_test_batch = next(test_generator)
X_test_batch_reshaped = X_test_batch.reshape(X_test_batch.shape[0], -1)
y_pred = clf.predict(X_test_batch_reshaped)

# Print the classification report
print(classification_report(y_test_batch, y_pred))


150 150


ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (750, 750, 3), y.shape = (241,)